#Part 1: Explore and cluster the neighborhoods in Toronto.
-----

In [2]:
import pandas as pd
import numpy as np

import requests
website_url=requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text

from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'lxml')
#print(soup.prettify())

In [3]:
My_table = soup.find('table',{'class':'wikitable sortable'})
#My_table

In [4]:
data = []
for tr in soup.tbody.find_all('tr'):
    data.append([ td.get_text().strip() for td in tr.find_all('td')])

In [5]:
df = pd.DataFrame(data, columns=['PostalCode','Borough','Neighborhood'])
df.head()

,PostalCode,Borough,Neighborhood
0,None,None,None
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


In [6]:
df.shape

(289, 3)

####Remove rows where Borough=None (Basically the first row.)

In [7]:
df.dropna(subset=["Borough"],axis=0,inplace=True)
df.head()

,PostalCode,Borough,Neighborhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


####Remove rows where Borough is not assigned.


In [8]:
df=df[df.Borough!="Not assigned"]
df.head(10)

,PostalCode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Not assigned
11,M9A,Etobicoke,Islington Avenue
12,M1B,Scarborough,Rouge
13,M1B,Scarborough,Malvern


####When the neighborhood is not assigned-> Neiborhood=Borough

In [9]:
for index, row in df.iterrows():
    if row['Neighborhood']=="Not assigned":
        df=df.replace(row['Neighborhood'],row['Borough'])
df.head(10)

,PostalCode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Queen's Park
11,M9A,Etobicoke,Islington Avenue
12,M1B,Scarborough,Rouge
13,M1B,Scarborough,Malvern


####Group by PostalCode (with neighborhoods separated by comma)

In [10]:
grp=df.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
grp.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [11]:
grp.shape

(103, 3)

#Part 2: Get the latitude and the longitude coordinates of each neighborhood.

In [12]:
#!pip install geocoder

In [13]:
import geocoder

In [14]:
'''def get_latlng(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    return lat_lng_coords
    
get_latlng('M5G')'''

"def get_latlng(postal_code):\n    # initialize your variable to None\n    lat_lng_coords = None\n    # loop until you get the coordinates\n    while(lat_lng_coords is None):\n        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))\n        lat_lng_coords = g.latlng\n    return lat_lng_coords\n    \nget_latlng('M5G')"

"Given that this package can be very unreliable, in case you are not able to get the geographical coordinates of the neighborhoods using the Geocoder package, here is a link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data"

In [15]:
df2=pd.read_csv("Geospatial_Coordinates.csv")
df2.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [16]:
df2.columns=['PostalCode','Latitude','Longtitude']
df2.head()

,PostalCode,Latitude,Longtitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [17]:
df_m=pd.merge(grp,df2,on="PostalCode")
df_m.head()

,PostalCode,Borough,Neighborhood,Latitude,Longtitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [19]:
df_m.shape

(103, 5)

In [23]:
df_m.to_csv(r'C:\Users\Dimitra\Desktop\work\github\Coursera_Capstone\latlon.csv')